# Django视图与网址进阶



## 一 、在网页上做加减法

### 1. 采用 /add/?a=4&b=5 这样GET方法进行
首先建一个新项目

```
$django admin startproject zqxt_views
```

然后在zqxt_views文件夹下创建一个app--calc

```
django-admin startapp calc
```

结构试图如下：
```
.
├── calc
│   ├── admin.py
│   ├── __init__.py
│   ├── migrations
│   │   └── __init__.py
│   ├── models.py
│   ├── tests.py
│   └── views.py
├── manage.py
└── zqxt_views
    ├── __init__.py
    ├── settings.py
    ├── urls.py
    └── wsgi.py
```

修改一下 calc/views.py文件

```python
from django.shortcuts import render
from django.http import HttpResponse
 
def add(request):
    a = request.GET['a']
    b = request.GET['b']
    c = int(a)+int(b)
    return HttpResponse(str(c))
```

修改 zqxt_views/urls.py 文件，添加一个网址来对应我们刚才新建的视图函数。

```python
from django.conf.urls import url
from django.contrib import admin
from calc import views as calc_views
 
 
urlpatterns = [
    url(r'^add/', calc_views.add, name='add'),  # 注意修改了这一行
    url(r'^admin/', admin.site.urls),
]
```

然后打开在终端打开服务器：

```python
$python manage.py runserver
```

这时候访问 http://127.0.0.1:8000/add/?a=4&b=5 会在页面看到一个数字9

### 2. 采用 /add/3/4/ 这样的网址的方式

修改 calc/views.py文件，再新定义一个add2 函数:

```python
def add2(request, a, b):
    c = int(a) + int(b)
    return HttpResponse(str(c))
```

修改 zqxt_views/urls.py 文件，再添加一个新的 url

```
url(r'^add/(\d+)/(\d+)/$', calc_views.add2, name='add2'),
```

然后访问 http://127.0.0.1:8000/add/4/5/ 就可以看到和刚才同样的效果，但是这回网址更优雅了

## 二、url 中的 name [技能提升]

urls.py 中的代码

```python
from django.conf.urls import url
from django.contrib import admin
from calc import views as calc_views
 
 
urlpatterns = [
    url(r'^add/', calc_views.add, name='add'),
    url(r'^add2/(\d+)/(\d+)/$', calc_views.add2, name='add2'),
    url(r'^admin/', admin.site.urls),
]
```

** name="add"是干什么呢？ **

我们在开发的时候，刚开始想用的是 /add2/4/5/ ，后来需求发生变化，比如我们又想改成 /4_add_5/这样的格式，但是我们在网页中，代码中很多地方都写死的是 

```html
<a href="/add2/4/5/">计算 4+5</a>
```

>这样就导致当我们改了 urls.py 后，对应的模板，甚至还视图中的跳转，以及 models.py 中也可能有获取网址的地方。

>每个地方都要改，修改的代价很大，一不小心，有的地方没改过来，那个就不能用了。

下面我们就用一种更加优雅的方式来解决

用 Python 代码获取对应的网址：

```python
>>> from django.core.urlresolvers import reverse
>>> reverse('add2', args=(4,5))
u'/add2/4/5/'
>>> reverse('add2', args=(444,555))
u'/add2/444/555/'
```
reverse 接收 url 中的 name 作为第一个参数，我们在代码中就可以通过 reverse() 来获取对应的网址（这个网址可以用来跳转，也可以用来计算相关页面的地址），只要对应的 url 的name不改，就不用改代码中的网址。

在网页模板中也是一样，可以很方便的使用。

不带参数的：
{% url 'name' %}

带参数的：参数可以是变量名
{% url 'name' 参数 %}
 
例如：
```html
<a href="{% url 'add2' 4 5 %}">link</a>
```

上面的代码渲染成最终的页面是

```html
<a href="/add2/4/5/">link</a>
```
这样就可以通过 {% url 'add2' 4 5 %} 获取到对应的网址 /add2/4/5/

当 urls.py 进行更改，前提是不改 name（这个参数设定好后不要轻易改），获取的网址也会动态地跟着变，比如改成：
```python
url(r'^new_add/(\d+)/(\d+)/$', 'calc.views.add2', name='add2'),
```

> add2变成了new_add，但是后面的name='add2'没改，这时{% url 'add2' 4 5 %} 就会渲染对应的网址成 /new_add/4/5/

> reverse 函数也是一样，获取的时候会跟着变成新的网址，这样，在想改网址时只需要改 urls.py 中的正则表达式（url 参数第一部分），其它地方都“自动”跟着变了

** 另外，如何让以前的 /add2/3/4/自动跳转到新的网址呢？要知道Django不会帮你做这个，这个需要自己来写一个跳转方法： **

在 views.py 写一个跳转的函数：

```python
from django.http import HttpResponseRedirect
from django.core.urlresolvers import reverse
 
 
def old_add2_redirect(request, a, b):
    return HttpResponseRedirect(
        reverse('add2', args=(a, b))
    )
```

urls.py中：
```python
    url(r'^add2/(\d+)/(\d+)/$', calc_views.old_add2_redirect),
    url(r'^new_add/(\d+)/(\d+)/$', calc_views.add2, name='add2'),
```

这样，假如用户收藏夹中有/add2/4/5/ ，访问时就会自动跳转到新的 /new_add/4/5/ 了